# Composite Measure: Eigenvalues via a Variational Problem

__The implimentation of this method is done for the Cross-In-Plane geometry__

Let $\Omega$ be our usual domain with singular inclusions represented by a graph $\mathbb{G}$, let $\theta$ be a fixed quasi-momentum value, and $\tilde{\lambda}$ be our composite measure $\tilde{\lambda} = \lambda_2 + \mu$.
Consider the problem of determining $\omega^2$ such that

\begin{align*}
    \omega^2 &= \inf\left\{ \int_{\Omega} \lvert \nabla^{\theta}v \rvert^2 \ \mathrm{d}\tilde{\lambda}, \ v\in H^1_{\theta}\left(\Omega, \mathrm{d}\tilde{\lambda}\right), \ \middle\vert \ \lvert\lvert v \rvert\rvert_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} = 1, \right\}. \tag{1}
\end{align*}

Assuming that a minimiser $u$ of (1) exists, then we can show that it satisfies our bulk-PDEs/edge-ODEs system for the composite medium.
Therefore, we will look to solve the variational problem (1), assuming a minimiser exists, and read off the values of $\omega^2$.

__NOTE__: For $\theta=0$, the constant function $u=\frac{1}{\sqrt{3}}$ is the minimiser, giving $\omega^2=0$.

In [1]:
import numpy as np
from numpy import pi

from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)

import time
from datetime import datetime

## Constrained Minimisation Problem

We will focus on our Cross-In-The-Plane geometry again, but move the single vertex $v_0$ in the peroid cell to the corner at $(0,0)$.
This leaves us with one horizontal edge $I_h$ on which $y=0$, and one vertical edge $I_v$ on which $x=0$; the two edges are loops from $v_0$ to itself.

Once again we need to choose a basis to truncate and approximate the minimising function in.
We will avoid the use of a Fourier basis, as this will force our normal derivatives along the graph edges to coincide, which is _not_ required for solutions to (1), nor indeed functions that belong to $H^1_{\theta}\left(\Omega, \mathrm{d}\tilde{\lambda}\right)$.
Instead, take $M\in\mathbb{N}$ and use the 2D-polynomial basis $\{\phi_m\}_{m=0}^{M^2-1}$ where

\begin{align*}
    \phi_m(x,y) &= p_m x^{i_m}y^{j_m}, &\qquad\text{where } m = j_m + Mi_m, \ i,j\in\{0,...,M-1\}, \\
    p_m &= \sqrt{ \frac{(2i_m+1)(2j_m+1)}{2i_m + 2j_m +3} },
\end{align*}

the normalisation constants $p_m$ ensuring that $\lvert\lvert \phi_m \rvert\rvert_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} = 1$.
We also choose this notation so that, when we come to impliment this method, we can easily switch between a matrix and column-vector representation for this basis.

__NOTE__: The polynomial basis is _not_ automatically periodic, so we will need to add constraints to our variational problem to ensure that we still represent functions in $H^1_{\theta}\left(\Omega, \mathrm{d}\tilde{\lambda}\right)$.

Thus, we seek the solution of the problem

\begin{align*}
    \text{Minimise} \quad & \quad \int_{\Omega} \lvert \nabla^{\theta}u \rvert^2 \ \mathrm{d}\tilde{\lambda}, \\
    \text{Subject to} \quad & \lvert\lvert u \rvert\rvert_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} = 1, \\
    & \quad u(0,y) = u(1,y), \\
    & \quad u(x,0) = u(x,1).
\end{align*}

If we want to go a step further and find $\omega_n^2, u_n$ pairs such that

\begin{align*}
        \omega_n^2 &= \min\left\{ \int_{\Omega} \lvert \nabla^{\theta}u \rvert^2 \ \mathrm{d}\tilde{\lambda}, \ u\in H^1_{\theta}\left(\Omega, \mathrm{d}\tilde{\lambda}\right), \ \middle\vert \ \lvert\lvert u \rvert\rvert_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} = 1, \ u_n \perp u_l, \ 1\leq l<n, \right\}, \tag{2}
\end{align*}
given the $n-1$ previous solutions $\omega_l^2, u_l$ for $1\leq l<n$, we also need to add the orthogonality conditions into our constrained minimisation problem.

### Problem to Study

With this in mind, this notebook aims to determine the solution $u_n\in H^1_{\theta}\left(\Omega, \mathrm{d}\tilde{\lambda}\right)$ to the problem:

\begin{align*}
    \text{Minimise} \quad & \quad J[u] := \int_{\Omega} \lvert \nabla^{\theta}u \rvert^2 \ \mathrm{d}\tilde{\lambda}, \\
    \text{Subject to} \quad & \lvert\lvert u \rvert\rvert_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} = 1, \\
    & \quad u(0,y) = u(1,y), \\
    & \quad u(x,0) = u(x,1), \\
    & \quad u \perp u_l, \ 1\leq l<n, \tag{3}
\end{align*}
where $\omega_n^2 := J[u_n]$, and $u_l$ is the solution to (3) with $n=l$.
When $n=1$, we have no orthogonality constraints.
Perpendicular means perpendicular in the $L^2(\Omega, \mathrm{d}\tilde{\lambda})$-sense.

## Numerical Solution: Setup


### Polynomial Representation

Recall the 2D-polynomial basis $\{\phi_m\}_{m=0}^{M^2-1}$ where

\begin{align*}
    \phi_m(x,y) &= x^{i_m}y^{j_m}, &\qquad\text{where } m = j_m + Mi_m, \ i,j\in\{0,...,M-1\}, \\
\end{align*}

introduced in the previous section.
Note that $\phi_m$ is real-valued, so $\phi_m = \overline{\phi}_m$ for any $m$.
Computing some relevant integrals provides us with the values of inner products between the polynomial functions; in these identities we can save on several cases by using a notion of "over-riding" division by 0.
That is, in any of the following formulae tagged with (ord), we interpret division by 0 as resulting in 0 - this saves us from having a large number of individual cases in our formulae, and is something we can actually exploit in our code.

__NOTE:__ We could normalise each $\phi_m$ using the prefactor $p_m = \sqrt{ \frac{(2i_m+1)(2j_m+1)}{2i_m + 2j_m +3} }$, however there isn't really a need since we don't have orthogonality of the $\phi_m$ anyway, so this isn't going to make computations any easier.
We will include these normalising constants in the following integral tables, and will compute them in the code which follows just in case they are required.

#### Integral Tables

For the part of the integrals with respect to $\lambda_2$, we have that

\begin{align*}
    \int_{\Omega} \phi_m\phi_n \ \mathrm{d}\lambda_2 
    &= \frac{p_m p_n}{(i_m+i_n+1)(j_m+j_n+1)}, \\
    \int_{\Omega} \mathrm{i}\theta\phi_m\cdot\nabla\phi_n \ \mathrm{d}\lambda_2 
    &= \mathrm{i}p_n p_m \left[ \frac{i_n\theta_1}{i_m+i_n}\frac{1}{j_m+j_n+1} + \frac{1}{i_m+i_n+1}\frac{j_n\theta_2}{j_m+j_n} \right], \tag{ord} \\
    \int_{\Omega} \nabla\phi_m\cdot\nabla\phi_n \ \mathrm{d}\lambda_2
    &= p_m p_n \left(\frac{i_m i_n}{i_m+i_n-1}\frac{1}{j_n+j_m+1} + \frac{j_m j_m}{j_m+j_n-1}\frac{1}{i_m+i_m+1}\right), \tag{ord} \\
    \int_{\Omega} \nabla^{\theta}\phi_m\cdot\overline{\nabla^{\theta}\phi_n} \ \mathrm{d}\lambda_2
    &= \int_{\Omega} \nabla\phi_m\cdot\nabla\phi_n + \mathrm{i}\theta\phi_m\cdot\nabla\phi_n - \mathrm{i}\theta\phi_n\cdot\nabla\phi_m + \lvert\theta\rvert^2\phi_m\phi_n \ \mathrm{d}\lambda_2 \\
    &= p_m p_n \left( \frac{i_m i_n}{i_m+i_n-1}\frac{1}{j_n+j_m+1} + \frac{j_m j_m}{j_m+j_n-1}\frac{1}{i_m+i_m+1} \right. \\
    & \quad \left. + \frac{\mathrm{i}\theta_1(i_n-i_m)}{i_m+i_n}\frac{1}{j_m+j_n+1} + \frac{\mathrm{i}\theta_2(j_n-j_m)}{j_m+j_n}\frac{1}{i_m+i_n+1} + \lvert\theta\rvert^2\frac{1}{i_m+i_n+1}\frac{1}{j_m+j_n+1} \right). \tag{ord}
\end{align*}

For any integrals on the edge $I_h$, we observe that $\phi_m=0$ unless $m = i_m M$, since $y=0$ on $I_h$.
Thus, we observe that

\begin{align*}
    \int_{I_h} \phi_{i_mM}\phi_{i_nM} \ \mathrm{d}\lambda_{h} 
    &= \frac{p_m p_n}{i_m+i_n+1}, \\
    \mathrm{i}\theta_{h}\int_{I_h} \phi_{i_mM}\phi_{i_nM}' \ \mathrm{d}\lambda_{h} 
    &= \frac{\mathrm{i}\theta_h p_m p_n i_n}{i_m+i_n}, \tag{ord} \\
    \int_{I_h} \phi_{i_mM}'\phi_{i_nM}' \ \mathrm{d}\lambda_{h} 
    &= \frac{p_m p_n i_m i_n}{i_m+i_n-1}, \tag{ord} \\
    \int_{I_h} \nabla^{\theta}\phi_{i_mM}\cdot\overline{\nabla^{\theta}\phi_{i_nM}} \ \mathrm{d}\lambda_h
    &= \int_{I_h} \phi_{i_mM}'\phi_{i_nM}' + \mathrm{i}\theta_{h}\phi_{i_mM}\phi_{i_nM}' - \mathrm{i}\theta_{h}\phi_{i_nM}\phi_{i_mM}' + \lvert\theta_h\rvert^2\phi_{i_mM}\phi_{i_nM} \ \mathrm{d}\lambda_{h} \\
    &= p_m p_n \left( \frac{i_m i_n}{i_m+i_n-1} + \frac{\mathrm{i}\theta_h(i_n-i_m)}{i_m+i_n} + \frac{\theta_h^2}{i_m+i_n+1} \right). \tag{ord}
\end{align*}

We obtain similar formulae for the vertical edge $I_v$; although here we must have $0\leq m = j_m\leq M-1$ for $\phi_m$ to be non-vanishing.

\begin{align*}
    \int_{I_v} \phi_{j_m}\phi_{j_n} \ \mathrm{d}\lambda_{v} 
    &= \frac{p_m p_n}{j_m+j_n+1}, \\
    \mathrm{i}\theta_{v}\int_{I_v} \phi_{j_m}\phi_{j_n}' \ \mathrm{d}\lambda_{v} 
    &= \frac{\mathrm{i}\theta_v p_m p_n j_n}{j_m+j_n}, \tag{ord} \\
    \int_{I_v} \phi_{j_m}'\phi_{j_n}' \ \mathrm{d}\lambda_{v} 
    &= \frac{p_m p_n j_m j_n}{j_m+j_n-1}, \tag{ord} \\
    \int_{I_v} \nabla^{\theta}\phi_{j_m}\cdot\overline{\nabla^{\theta}\phi_{j_n}} \ \mathrm{d}\lambda_v
    &= \int_{I_v} \phi_{j_m}'\phi_{j_n}' + \mathrm{i}\theta_{v}\phi_{j_m}\phi_{j_n}' - \mathrm{i}\theta_{v}\phi_{j_n}\phi_{j_m}' + \lvert\theta_v\rvert^2\phi_{j_m}\phi_{j_m} \ \mathrm{d}\lambda_{v} \\
    &= p_m p_n \left( \frac{j_m j_n}{j_m+j_n-1} + \frac{\mathrm{i}\theta_v(j_n-j_m)}{j_m+j_n} + \frac{\theta_v^2}{j_m+j_n+1} \right). \tag{ord}
\end{align*}

From these, we can then compute $\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})}$ by taking appropriate sums.
The function below will perform the "overide divison" operation that we tag with (ord).

In [2]:
def DivOveride(a, b):
    '''
    Performs our "override division" operation a/b, where we interpret division by 0 as returning 0.
    INPUTS:
        a: (n,), numerator array or scalar
        b: (n,), denominator array or scalar
    OUTPUTS:
        quo: (n,) float, result of division a/b and setting division by 0 to result in 0
    '''
    return np.divide(a, b, out=np.zeros_like(b, dtype=float), where=b!=0)

#### Inner Product Stores

At this point, it is helpful for us to build matrices that can store the various "inner products" (results of the formulae above) so that we can save on computations in the long run.

We begin by computing and storing the parts of the inner products pertaining to $\lambda_2$.

In [3]:
def Lambda2Stores(M, theta):
    '''
    Builds matrices that will store the various inner products wrt lambda_2 between the basis functions, 
    so that our later computations do not have to.
    INPUTS:
        M: int, M-1 is the highest order of the polynomial basis we are using
        theta: (2,) float, value of the quasi-momentum
    OUTPUTS: 
    All are shape (M^2-1,M^2-1) and list inner products wrt lambda_2 only.
        pmpn: float, pmpn[m,n] = p+m*p_n
        l2mn: float, l2mn[m,n] = <phi_m,phi_n> / pmpn
        l2mdn: float, l2mdn[m,n] = <i\theta phi_m, \grad phi_n> / (pmpn*1.j)
        l2dmdn: float, l2dmdn[m,n] = <\grad phi_m, \grad phi_n> / pmpn
        l2dtmdtn: complex, l2dtmdtn[m,n] = <\grad^\theta phi_m, \grad^\theta phi_n> / pmpn
    '''

    # SETUP: Create array of combinations of i & j values for a given m
    
    iInds = jInds = np.arange(0,M,dtype=int)
    ijVals = np.array(np.meshgrid(iInds, jInds)).T.reshape(-1,2) # ijVals[m,:] = [i_m,j_m]
    # np.add.outer(a,b) returns an array AB with entries AB[m,n] = a[m]+b[n]
    Imn = np.add.outer(ijVals[:,0],ijVals[:,0]) # Imn[m,n] = i_m+i_n
    Jmn = np.add.outer(ijVals[:,1],ijVals[:,1]) # Jmn[m,n] = j_m+j_n
    
    # first, we build a vector of the p_m
    # p_m = \sqrt{ \frac{(2i_m+1)(2j_m+1)}{2i_m + 2j_m +3} }
    pm = (2*ijVals[:,0] + 1)*(2*ijVals[:,1]+1) / ( 2*ijVals[:,0] + 2*ijVals[:,1] + 3 )
    pm = np.sqrt(pm)
    
    # next, we need to compute the various inner products
    # we'll store these in a matrix of shape (M^2-1,M^2-1,l) where
    # l is the number of inner products that we want to store
    # details will be given below
    
    pmpn = np.outer(pm,pm)
    # pmpn[m,n] = pm * pn
    # this also saves us from appending pmpn to all our following calculations!
    
    # stores at index (m,n) the value <phi_m,phi_n>_{lambda2} / pmpn
    # note: l2mn is symmetric!
    l2mn = (1/(Imn+1)) * (1/(Jmn+1))
    
    # stores at index (m,n) the value <i\theta phi_m, \grad phi_n>_{lambda2} / (pmpn*1.j)
    # NOTE: the decision to leave out the i and store this as a float to save memory space!
    l2mdn = theta[0]*(DivOveride(1,Imn)*ijVals[:,0])/(Jmn+1) + theta[1]*(DivOveride(1,Jmn)*ijVals[:,1])/(Imn+1)
    
    # stores at index (m,n) the value <\grad phi_m, \grad phi_n>_{lambda2} / pmpn
    # note: l2dmdn is symmetric!
    l2dmdn = DivOveride( np.outer(ijVals[:,0], ijVals[:,0]), Imn-1 ) * (1/(Jmn+1))
    l2dmdn += DivOveride( np.outer(ijVals[:,1], ijVals[:,1]), Jmn-1 ) * (1/(Imn+1))
    
    # stores at index (m,n) the value <\grad^\theta phi_m, \grad^\theta phi_n>_{lambda2} / pmpn
    # note: l2mdn.T "=" l2ndm
    l2dtmdtn = l2dmdn + 1.j*l2mdn.T - 1.j*l2mdn + np.dot(theta, theta)*l2mn
    
    return pmpn, l2mn, l2mdn, l2dmdn, l2dtmdtn

Next, we can create stores for the inner products with respect to $\lambda_h$, the singular measure along the horizontal edge.

In [4]:
def LambdaHStores(M, theta):
    '''
    Builds matrices that will store the various inner products wrt lambda_h between the basis functions, 
    so that our later computations do not have to.
    INPUTS:
        M: int, M-1 is the highest order of the polynomial basis we are using
        theta: (2,) float, value of the quasi-momentum
    OUTPUTS: 
    All are shape (M^2-1,M^2-1) and list inner products wrt lambda_h only.
    pmpn[m,n] = p_m * p_n throughout.
        lhmn: float, lhmn[m,n] = <phi_m,phi_n> / pmpn
        lhmdn: float, lhmdn[m,n] = <i\theta_h phi_m, \grad phi_n> / (pmpn*1.j)
        lhdmdn: float, lhdmdn[m,n] = <\grad phi_m, \grad phi_n> / pmpn
        lhdtmdtn: complex, lhdtmdtn[m,n] = <\grad^\theta phi_m, \grad^\theta phi_n> / pmpn
    '''
    
    # SETUP: Create array of combinations of i & j values for a given m
    
    iInds = jInds = np.arange(0,M,dtype=int)
    ijVals = np.array(np.meshgrid(iInds, jInds)).T.reshape(-1,2) # ijVals[m,:] = [i_m,j_m]
    # np.add.outer(a,b) returns an array AB with entries AB[m,n] = a[m]+b[n]
    Imn = np.add.outer(ijVals[:,0],ijVals[:,0]) # Imn[m,n] = i_m+i_n
    Jmn = np.add.outer(ijVals[:,1],ijVals[:,1]) # Jmn[m,n] = j_m+j_n
    
    # throughout, we only need to do these for when m = i_m M and n = i_n M
    # as all other IP's are 0 on this edge.
    # Jmn[m,n] > 0 <=> j_m or j_n != 0, so this can be used to determine where we should leave 0's
    
    # stores at index (m,n) the value <phi_m,phi_n> / pmpn
    lhmn = np.divide(1, Imn+1, out=np.zeros_like(Imn, dtype=float), where=Jmn==0)
    
    # stores at index (m,n) the value <i\theta_h phi_m, \grad phi_n> / (pmpn*1.j)
    # QM on this edge is theta[0]
    lhmdn = theta[0] * DivOveride(1, Imn) * ijVals[:,0]
    lhmdn[Jmn>0] = 0. #set the values we didn't need to look at to be 0
    
    # stores at index (m,n) the value <\grad phi_m, \grad phi_n> / pmpn
    lhdmdn = DivOveride( np.outer(ijVals[:,0], ijVals[:,0]), Imn-1 )
    lhdmdn[Jmn>0] = 0. #set the values we didn't need to look at to be 0
    
    # stores at index (m,n) the value <\grad^\theta phi_m, \grad^\theta phi_n> / pmpn
    lhdtmdtn = lhdmdn + 1.j*lhmdn.T - 1.j*lhmdn + np.dot(theta[0], theta[0])*lhmn
    
    return lhmn, lhmdn, lhdmdn, lhdtmdtn

And finally, stores for the inner products along the vertical edge.

In [5]:
def LambdaVStores(M, theta):
    '''
    Builds matrices that will store the various inner products wrt lambda_v between the basis functions, 
    so that our later computations do not have to.
    INPUTS:
        M: int, M-1 is the highest order of the polynomial basis we are using
        theta: (2,) float, value of the quasi-momentum
    OUTPUTS: 
    All are shape (M^2-1,M^2-1) and list inner products wrt lambda_v only.
    pmpn[m,n] = p_m * p_n throughout.
        lvmn: float, lvmn[m,n] = <phi_m,phi_n> / pmpn
        lvmdn: float, lvmdn[m,n] = <i\theta_v phi_m, \grad phi_n> / (pmpn*1.j)
        lvdmdn: float, lvdmdn[m,n] = <\grad phi_m, \grad phi_n> / pmpn
        lvdtmdtn: complex, lvdtmdtn[m,n] = <\grad^\theta phi_m, \grad^\theta phi_n> / pmpn
    '''
    
    # SETUP: Create array of combinations of i & j values for a given m
    
    iInds = jInds = np.arange(0,M,dtype=int)
    ijVals = np.array(np.meshgrid(iInds, jInds)).T.reshape(-1,2) # ijVals[m,:] = [i_m,j_m]
    # np.add.outer(a,b) returns an array AB with entries AB[m,n] = a[m]+b[n]
    Imn = np.add.outer(ijVals[:,0],ijVals[:,0]) # Imn[m,n] = i_m+i_n
    Jmn = np.add.outer(ijVals[:,1],ijVals[:,1]) # Jmn[m,n] = j_m+j_n
    
    # throughout, we only need to do these for when m = i_m M and n = i_n M
    # as all other IP's are 0 on this edge.
    # Jmn[m,n] > 0 <=> j_m or j_n != 0, so this can be used to determine where we should leave 0's
    
    # stores at index (m,n) the value <phi_m,phi_n> / pmpn
    lvmn = np.divide(1, Jmn+1, out=np.zeros_like(Jmn, dtype=float), where=Imn==0)
    
    # stores at index (m,n) the value <i\theta_h phi_m, \grad phi_n> / (pmpn*1.j)
    # QM on this edge is theta[1]
    lvmdn = theta[1] * DivOveride(1, Jmn) * ijVals[:,1]
    lvmdn[Imn>0] = 0. #set the values we didn't need to look at to be 0
    
    # stores at index (m,n) the value <\grad phi_m, \grad phi_n> / pmpn
    lvdmdn = DivOveride( np.outer(ijVals[:,1], ijVals[:,1]), Jmn-1 )
    lvdmdn[Imn>0] = 0. #set the values we didn't need to look at to be 0
    
    # stores at index (m,n) the value <\grad^\theta phi_m, \grad^\theta phi_n> / pmpn
    lvdtmdtn = lvdmdn + 1.j*lvmdn.T - 1.j*lvmdn + np.dot(theta[1], theta[1])*lvmn
    
    return lvmn, lvmdn, lvdmdn, lvdtmdtn

And with these, we can then obtain the inner products with respect to $\tilde{\lambda}$.

In [6]:
def TLambdaStores(M, theta):
    '''
    Computes the matrices whose (m,n)-th entries contain the inner products 
    <\grad^\theta phi_m, \grad^\theta phi_n>_{\compMes} and
    < phi_m, phi_n>_{\compMes}.
    INPUTS:
        M: int, M-1 is the highest order of the polynomial basis we are using
        theta: (2,) float, value of the quasi-momentum
    OUTPUTS:
    All are shape (M^2-1,M^2-1) and list inner products wrt \tilde{\lambda} only.
        ip: float, ip[m,n] = < phi_m, phi_n>_{\compMes}
        ipDT: complex, ipD[m,n] =  <\grad^\theta phi_m, \grad^\theta phi_n>_{\compMes}
    '''
    
    # bulk terms
    pmpn, l2mn, _, _, l2dtmdtn = Lambda2Stores(M, theta)
    # horz edge terms
    lhmn, _, _, lhdtmdtn = LambdaHStores(M, theta)
    # vert edge terms
    lvmn, _, _, lvdtmdtn = LambdaVStores(M, theta)
    
    # now, reapply pmpn to each term and construct the norms in the whole space
    # remember, we need to replace pmpn in our terms now
    # NOTE: If we aren't treating p_m=1 for each pm, you'll need to uncomment the multiplication by pmpn
    ip = ( l2mn + lhmn + lvmn ) #* pmpn
    ipDT = ( l2dtmdtn + lhdtmdtn + lvdtmdtn ) #* pmpn
    return ip, ipDT

## Discretisation

We elect to represent the solution $u_n$ to (3) as

\begin{align*}
    u_n(x,y) &= \sum_{m=0}^{M^2-1} u_m\phi_m(x,y), \qquad u_m\in\mathbb{C}.
\end{align*}

This means that we are going to be optimising over the complex coefficients $u_m$.
We write $U=(u_1,...,u_m)^\top$ for ease in what follows.

At this point, we also have to remember that `SciPy` cannot optimise over complex arguments, and thus we need to treat this problem using the vector $UU$, where `UU[0::2] = np.real(U)` and `UU[1::2] = np.imag(U)`.
As such, we'll need to use the converter functions:

In [7]:
def Real2Comp(x):
    '''
    Given a vector x of length (2n,), which stores the real and imaginary parts of complex numbers z as
    z[j] = x[2j] + i*x[2j+1], return the vector z.
    INPUTS:
        x: (2n,) float, real and imaginary parts of a vector of complex numbers z
    OUTPUTS:
        z: (n,) complex, z[j] = x[2j] + i*x[2j+1]
    '''
    
    z = np.zeros((len(x)//2,), dtype=complex)
    z = x[np.arange(0, len(x), 2)] + 1.j*x[np.arange(1, len(x), 2)]
    return z

def Comp2Real(z):
    '''
    Given a vector z of length (n,) of complex numbers, return a real array x where
    z[j] = x[2j] + i*x[2j+1]
    '''
    
    x = np.zeros((2*len(z),), dtype=float)
    x[np.arange(0, len(x), 2)] = np.real(z)
    x[np.arange(1, len(x), 2)] = np.imag(z)
    return x

### The objective function $J$

Re-expressing the functional $J$ in terms of the $u_m$ and the basis $\phi_m$, we obtain that

\begin{align*}
    J[U] &= \sum_{m=0}^{M^2-1}\sum_{n=0}^{M^2-1} u_m\overline{u}_n \langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})},
\end{align*}

where we can compute the inner product from the terms above.
Indeed, we can compute $J[U]$ by
```python
JU = np.sum( np.outer(U, np.conjugate(U)) * ipDT)
```
where `ipDT[m,n]`$=\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})}$, which we can get from our builder functions above.

We can also compute that

\begin{align*}
    \dfrac{\partial J}{\partial \Re(u_m)} &= 2\sum_{n=0}^{M^2-1} \Re(u_n)\Re\left(\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right) + \Im(u_n)\Im\left(\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right), \\
    \dfrac{\partial J}{\partial \Im(u_m)} &= 2\sum_{n=0}^{M^2-1}\Im(u_n)\Re\left(\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right) - \Re(u_n)\Im\left(\langle \nabla^{\theta}_{\tilde{\lambda}}\phi_m, \nabla^{\theta}_{\tilde{\lambda}}\phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right),   
\end{align*}

which means that we can relatively easily compute the Jacobian on $J$ too, via
```python
jac[0::2] = 2 * ( np.matmul( np.real(ipDT), UU[0::2] ) + np.matmul( np.imag(ipDT), UU[1::2] ) )
jac[1::2] = 2 * ( np.matmul( np.real(ipDT), UU[1::2] ) - np.matmul( np.imag(ipDT), UU[0::2] ) )
```

In [8]:
def J(UU, ipDT):
    '''
    Evaluates the objective function J as a function of UU.
    INPUTS:
        UU: (2M,) float, representing the coefficient vector. Should be the case that U = Real2Comp(UU).
        ipDT: (M^2-1,M^2-1) float, matrix where entry (m,n) is <\grad^\theta phi_m, \grad^\theta phi_n>_{\compMes}.
    OUTPUTS:
        Jval: float, the value of the objective function. By definition, this is real.
    '''
    
    U = Real2Comp(UU)
    # casting to real is safe, since this expression is always real by definition
    Jval = np.real( np.sum( np.outer(U, np.conjugate(U)) * ipDT ) )
    return Jval

def JandJac(UU, ipDT):
    '''
    Evaluates the objective function J and it's Jacobian as a function of UU, returning them as a tuple.
    INPUTS:
        UU: (2M,) float, representing the coefficient vector. Should be the case that U = Real2Comp(UU).
        ipDT: (M^2-1,M^2-1) float, matrix where entry (m,n) is <\grad^\theta phi_m, \grad^\theta phi_n>_{\compMes}.
    OUTPUTS:
        Jval: float, the value of the objective function. By definition, this is real.
        JacVal: (2M,) float, value of the jacobian of the objective function, which is again real by definition
    '''
    
    U = Real2Comp(UU)
    # casting to real is safe, since this expression is always real by definition
    Jval = np.real( np.sum( np.outer(U, np.conjugate(U)) * ipDT ) )
    
    JacVal = np.zeros_like(UU, dtype=float)
    # shouldn't have to cast to real since we only deal with real and imag parts anyway
    JacVal[0::2] = 2. * ( np.matmul(np.real(ipDT),UU[0::2]) + np.matmul(np.imag(ipDT),UU[1::2]) )
    JacVal[1::2] = 2. * ( np.matmul(np.real(ipDT),UU[1::2]) - np.matmul(np.imag(ipDT),UU[0::2]) )
    
    return Jval, JacVal

### The periodicity constraints

The requirement that $u$ be periodic, namely that $u(0,y)=u(1,y)$ and $u(x,0)=u(x,1)$ amounts to comparing the coefficients in two polynomials, and forcing these to be equal.
The downside of this is that our constraints matrix rapidly becomes bloated with these comparisons, but nonetheless we have to proceed.

We have that

\begin{align*}
    u(0,y) = \sum_{j_m=0}^{M-1} u_{j_m}y^{j_m}, \qquad
    u(1,y) = \sum_{j_m=0}^{M-1} y^{j_m}\left( \sum_{i_m=0}^{M-1} u_{j_m+i_mM} \right),
\end{align*}

and therefore require that

\begin{align*}
    0 &= \sum_{j_m}^{M-1}y^{j_m}\left( \sum_{i_m=1}^{M-1} u_{j_m+i_mM} \right).
\end{align*}

Note the starting index of 1 now for $i_m$.
The coefficient of $y^{j_m}$ must be zero for every $j_m$, and thus we obtain $M$ constraints:

\begin{align*}
    \sum_{i_m=1}^{M-1} u_{j_m+i_mM} = 0, \qquad \forall j_m\in\{0,...,M-1\}.
\end{align*}

Similarly for the condition across $u(x,0)=u(x,1)$, we have a further $M$ constraints:

\begin{align*}
    \sum_{j_m=1}^{M-1} u_{j_m+i_mM} = 0, \qquad \forall i_m\in\{0,...,M-1\}.
\end{align*}

However, we must work in terms of `UU` and not $U$.
This isn't too bad though, as we just seperate the real and imaginary parts of the expression.
Indeed, we can see that

\begin{align*}
    \Re\left(\sum_{i_m=1}^{M-1} u_{j_m+i_mM}\right) = (0,...,0,1,0,...,0,1,0,...) UU,
\end{align*}

where the $1$s are placed at indices $2(j_m + i_mM), \ i_m\in\{1,...,M-1\}$.
Similarly,

\begin{align*}
    \Im\left(\sum_{i_m=1}^{M-1} u_{j_m+i_mM}\right) = (0,...,0,1,0,...,0,1,0,...) UU,
\end{align*}

where the $1$s are placed at indices $2(j_m + i_mM)+1, \ i_m\in\{1,...,M-1\}$.

For the other periodicity constraints, we find that

\begin{align*}
    \Re\left( \sum_{j_m=1}^{M-1} u_{j_m+i_mM} \right) &= (0,...,1,0,1,0,1,...,1,0,0,...)UU, \\
    \Im\left( \sum_{j_m=1}^{M-1} u_{j_m+i_mM} \right) &= (0,...,0,1,0,1,0,...,0,1,0,...)UU,
\end{align*}

where the $1$s are placed at indices (for $j_m\in\{1,...,M-1\}$) $2(i_m M + j_m)$ in the real part case and $2(i_m M + j_m)+1$ in the imaginary part case.

In [9]:
def PeriodicConstraints(M):
    '''
    Given the highest order polynomial we are using to approximate, create the constraints corresponding to the
    forcing of periodicity of the solution.
    INPUTS:
        M: int, M-1 is the highest power in the polynomial approximation we are using.
    OUTPUTS:
        ConstraintMatrix: (4M,2M^2) int, the matrix defining the linear constraint.
    '''
    
    # we will have 4M conditions: there are 2 periodicity conditions giving us M equations each,
    # and then we have to split into real and imaginary parts
    # slice convention: start:stop:step, where we DONT INCLUDE stop!
    
    # on the plus side, our constraint matrix is a matrix of integers!
    # 4M constraints (rows) relating 2M^2 (size of UU) variables (columns)
    ConstraintMatrix = np.zeros((4*M, 2*M*M), dtype=int)

    # these are the periodicity constraints along the x-boundaries
    for jm in range(M):
        ##REAL PART
        # 1st one appears at index 2(jm + M)
        # go up in steps of size 2M
        # last one appears at index 2(jm + M(M-1))
        ConstraintMatrix[2*jm , 2*(jm+M):2*(jm+M*(M-1))+1:2*M] = np.ones((M-1), dtype=int)
        
        ##IMAG PART
        # 1st one appears at index 2(jm + M)+1
        # go up in steps of size 2M
        # last one appears at index 2(jm + M(M-1))+1      
        ConstraintMatrix[2*jm+1 , 2*(jm+M)+1:2*(jm+M*(M-1))+2:2*M] = np.ones((M-1), dtype=int)
    
    # now for the periodicity constraints along the y-boundaries
    for im in range(M):
        ##REAL PART
        # 1st one appears at index 2(im*M + 1)
        # go up in steps of 2
        # last one appears at index 2(im*M + M-1)
        ConstraintMatrix[2*M + 2*im , 2*(im*M + 1):2*(im*M + M-1)+1:2] = np.ones((M-1), dtype=int)
        
        ##IMAG PART
        # 1st one appears at index 2(im*M + 1)+1
        # go up in steps of 2
        # last one appears at index 2(im*M + M-1)+1
        ConstraintMatrix[2*M + 2*im + 1 , 2*(im*M + 1)+1:2*(im*M + M-1)+2:2] = np.ones((M-1), dtype=int)

    return ConstraintMatrix

### Orthogonality Constraints

The condition that $u$ be orthogonal to the $l$ previous functions $u^l$ can also be expressed as a linear constraint.
Let `UUl` denote the (`Comp2Real`) vector of coefficients for the function $u^l$.
Then the condition that $u$ be orthogonal (in $L^2(\Omega, \mathrm{d}\tilde{\lambda})$) to $u_l$ can be expressed as two conditions (for each of the real and imaginary parts);

\begin{align*}
    0 &= \sum_{m=0}^{M^2-1} \Re(u_m) \left( \sum_{n=0}^{M^2-1} \Re(u^l_n) \langle \phi_m,\phi_n \rangle\right)
    + \sum_{m=0}^{M^2-1} \Im(u_m) \left( \sum_{n=0}^{M^2-1} \Im(u^l_n) \langle \phi_m,\phi_n \rangle\right), \\
    0 &= \sum_{m=0}^{M^2-1} \Im(u_m) \left( \sum_{n=0}^{M^2-1} \Re(u^l_n) \langle \phi_m,\phi_n \rangle\right)
    - \sum_{m=0}^{M^2-1} \Re(u_m) \left( \sum_{n=0}^{M^2-1} \Im(u^l_n) \langle \phi_m,\phi_n \rangle\right).
\end{align*}

In terms of our optimisation variables `UU`, we have the following syntax:

\begin{align*}
    0 &= \sum_{m=0}^{M^2-1} UU[2m] \left( \sum_{n=0}^{M^2-1} UUl[2n] ip[m,n]\right)
    + \sum_{m=0}^{M^2-1} UU[2m+1] \left( \sum_{n=0}^{M^2-1} UUl[2n+1] ip[m,n]\right), \\
    0 &= \sum_{m=0}^{M^2-1} UU[2m+1] \left( \sum_{n=0}^{M^2-1} UUl[2n] ip[m,n]\right)
    - \sum_{m=0}^{M^2-1} UU[2m] \left( \sum_{n=0}^{M^2-1} UUl[2n+1] ip[m,n]\right).
\end{align*}

In [10]:
def OrthogonalityConstraints(M, prevUs, ip):
    '''
    Given the highest order polynomial we are using to approximate, and the previous polynomials that
    we are required to be orthogonal to,
    create the constraints corresponding to the orthogonality of u to each of the u^l.
    INPUTS:
        M: int, M-1 is the highest power in the polynomial approximation we are using.
        prevUs: (l,2M^2) float, row p is the vector UUp for 1<=p<=l.
        ip: (M^2-1,M^2-1) float, matrix where entry (m,n) is <phi_m, phi_n>_{\compMes}.
    OUTPUTS:
        ConstraintMatrix: (2l,2M^2) float, the matrix defining the linear constraint.
    '''
    
    lMax = np.shape(prevUs)[0]
    ConstraintMatrix = np.zeros((2*lMax, 2*M*M), dtype=float)
    
    for l in range(lMax):
        # construct the rows that correspond to orthogonality to prevUs[l]
        UUl = prevUs[l,:]
        # this vector is such that pReal[m] = sum_n( UUl[2n]ip[m,n] )
        pReal = np.sum( UUl[0::2]*ip, axis=1 )
        # this vector is such that pImag[m] = sum_n( UUl[2n+1]ip[m,n] )
        pImag = np.sum( UUl[1::2]*ip, axis=1 )
        
        #REAL PART
        ConstraintMatrix[2*l,0::2] = pReal #UU[2m] has coefficient sum_n( UUl[2n]*ip[m,n] )
        ConstraintMatrix[2*l,1::2] = pImag #UU[2m+1] has coefficient sum_n( UUl[2n+1]*ip[m,n] )
        #IMAG PART
        ConstraintMatrix[2*l+1,0::2] = - pImag #UU[2m] has coefficient -sum_n( UUl[2n+1]ip[m,n] )
        ConstraintMatrix[2*l+1,1::2] = pReal ##UU[2m+1] has coefficient sum_n( UUl[2n]ip[m,n] )
        
    return ConstraintMatrix

#### LinearConstraint

We now simply append our two linear constraint matrices together to form the LinearConstraint object for the minimisation problem.

In [11]:
def BuildLinearConstraint(M, ip, prevUs=0):
    '''
    Build the linear constraint matrix for the minimisation problem.
    The first 4M rows are the periodicity constraints, and will be build by PeriodicConstraints.
    The remaining rows are the orthogonality conditions given the previous functions in prevUs,
    there will be no additional rows is prevUs is not provided (assuming no orthogonality conditions to be met).
    INPUTS:
        M: int, M-1 is the highest power in the polynomial approximation we are using.
        prevUs: (l,2M^2) float, row p is the vector UUp for 1<=p<=l.
        ip: (M^2-1,M^2-1) float, matrix where entry (m,n) is <phi_m, phi_n>_{\compMes}.
    OUTPUTS:
        LinearConstraint: scipy.optimize.LinearConstraint, specifying the periodicity and orthogonality constraints.
        constraintMatrix: (4M+2l, 2*M*M) float, the matrix specifying the constraint, for error checking purposes.
    '''
    
    MperiodicPart = PeriodicConstraints(M)
    if np.ndim(prevUs)==0:
        # if prevUs is only a scalar, then we haven't provided it (or have provided it improperly), 
        # so assume no orthogonality constraints
        constraintMatrix = MperiodicPart
    else:
        # we have some orthogonality conditions
        MorthPart = OrthogonalityConstraints(M, prevUs, ip)
        constraintMatrix = np.vstack((MperiodicPart, MorthPart))
    
    # note: lb can be a scalar if the bounds are all the same!
    return LinearConstraint(constraintMatrix, 0, 0), constraintMatrix

### The norm constraint

The requirement that the $L^2(\Omega, \mathrm{d}\tilde{\lambda})$-norm of $u$ be unity can be written as

\begin{align*}
    1 &= \sum_{m=0}^{M^2-1}\sum_{n=0}^{M^2-1} u_m \overline{u}_n \langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})},
\end{align*}

which is again easily computable as
```python
normU = np.sum( np.outer(U, np.conjugate(U)) * ip )
```

where `ip[m,n]`$=\langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})}$, which again we can get from our builder functions above.

__NOTE__: The norm constraint is a nonlinear constraint, so will need to be handled differently to the periodicity contraints, which are linear.

We can also compute that (by analogy with the objective function's expression)

\begin{align*}
    \dfrac{\partial }{\partial \Re(u_m)} &= 2\sum_{n=0}^{M^2-1} \Re(u_n)\Re\left(\langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right) + \Im(u_n)\Im\left(\langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right), \\
    \dfrac{\partial }{\partial \Im(u_m)} &= 2\sum_{n=0}^{M^2-1}\Im(u_n)\Re\left(\langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right) - \Re(u_n)\Im\left(\langle \phi_m, \phi_n \rangle_{L^2(\Omega, \mathrm{d}\tilde{\lambda})} \right),   
\end{align*}

which means that we can relatively easily compute the Jacobian of the norm constraint too, via
```python
jac[0,0::2] = 2 * ( np.matmul( np.real(ipDT), UU[0::2] ) + np.matmul( np.imag(ipDT), UU[1::2] ) )
jac[0,1::2] = 2 * ( np.matmul( np.real(ipDT), UU[1::2] ) - np.matmul( np.imag(ipDT), UU[0::2] ) )
```

In [12]:
def NormConstraint(M, ip, giveJac=True):
    '''
    Given the highest order polynomial we are using to approximate, create the constraints corresponding to the
    forcing of the norm of the solution to be one.
    INPUTS:
        M: int, M-1 is the highest power in the polynomial approximation we are using.
        ip: (M^2-1,M^2-1) float, matrix where entry (m,n) is < phi_m, phi_n>_{\compMes}.
    OUTPUTS:
        NonlinearConstraint: scipy.optimize.NoninearConstraint, defines the norm constraint.   
    '''
    
    # we must construct a function f(UU) such that
    # lb <= f(UU) <= ub
    # is the form for our nonlinear constraints
    def f(UU):
        '''
        Computes the vector of constraints at the given value of UU
        '''
        
        U = Real2Comp(UU)
        # cast to real should be safe, since this expression is always real by definition
        return np.real( np.sum( np.outer(U, np.conjugate(U)) * ip ) )
    
    # also provide the jacobian of this constraint
    def Jf(UU):
        '''
        Computes the Jacobian of the vector of constraints at the given value of UU
        '''
        
        JacVal = np.zeros_like(UU, dtype=float)
        # shouldn't have to cast to real since we only deal with real and imag parts anyway
        JacVal[0::2] = 2. * ( np.matmul(np.real(ip),UU[0::2]) + np.matmul(np.imag(ip),UU[1::2]) )
        JacVal[1::2] = 2. * ( np.matmul(np.real(ip),UU[1::2]) - np.matmul(np.imag(ip),UU[0::2]) )
        # return as a (2M,) vector
        return JacVal#.reshape((1,np.shape(UU)[0]))
    
    # the lower bound and the upper bound coincide at 1, so just provide scalars here
    if giveJac:
        return NonlinearConstraint(f, 1, 1, jac=Jf), f
    else:
        return NonlinearConstraint(f, 1, 1), f

### Saving the Resulting Eigenfunctions

We will also need a class to store our "eigenfunctions" in, so that they can easily be evaluated and plotted.
This is the class `Poly2D`, which we define below.

__NOTE:__ This class must appear _after_ we define the objective function, as it's definition depends on that!
Better programatic practice would be to define the function itself within the class.

#### Attributes:
- `uCoeffs`: The coefficients $u_m$ in the representation $u = \sum_{m=1}^{M^2-1} u_m\phi_m$.
- `M`: $M-1$ is the highest order of the polynomial representation that we are going to.
- `theta`: $\theta$ is a `(2,)` float vector containing the quasi-momentum value.
- `lbda`: `lbda`$=\omega^2$ is the eigenvalue corresponding to this polynomial.

#### Methods:
- `__init__(theta, U)`: Initialisation method for a class instance. `theta` is set from the input, and `U` is set to be the attribute `uCoeffs`, and used to compute what `M` must be. `lbda` is determined from then evaluating the objective function `_J`.
- `val(x,y)`: Given two arrays of equal length containing $(x,y)$ coordinates when paired up, evaluates $u$ at these points. Is vectorised and compatable with `np.meshgrid`.
- `plot(N, levels)`: Creates a plot of the function $u$ over the domain $\left[0,1\right)^2$, using $N$ meshpoints in each direction and `levels` contour levels.

In [13]:
class Poly2D:
    '''
    Stores 2D polynomials of the form
        u(x,y) = \sum_{m=0}^{M^2-1} u_m\phi_m,
    for the basis functions phi_m defined above.
    ATTRIBUTES:
        uCoeffs: (M*M,) complex, the coefficients in the representation
        M: int, M-1 is the highest-order term of the polynomial
        theta: (2,) float, the value of the quasi-momentum that this polynomial was computed at
        lbda: float, the "eigenvalue" (or value of the objective function) corresponding to this polynomial
    METHODS:
        val(x,y): evaluates the polynomial the the coordinate pairs (x,y)
    '''
      
    # initialisation method
    def __init__(self, theta, U):
        '''
        Initialisation method for an instance of the class.
        INPUTS:
            theta: (2,) float, the value of the quasi-momentum that this polynomial was computed at
            U: (M*M,) complex, the coefficients in the representation of the function
        OUTPUTS:
            Poly2D: instance of class, the values of M and lbda are set automatically
        '''
        
        # QM is given to us for free
        self.theta = theta
        # as is the vector of coefficients
        self.uCoeffs = U
        # can deduce M from the length of U
        self.M = int( np.sqrt(np.shape(U)[0]) )
        # finally, record the value of lbda
        self.lbda = self._J(U)
        # we are now all done
        return
    
    def __str__(self):
        '''
        Default print output when instance of class is passed to print()
        '''
        rFig, iFig = self.plot()
        rFig[0].show()
        iFig[0].show()
        return 'Poly2D with M = %d' % self.M
    
    def _J(self, U):
        '''
        Returns the value of lbda=omega^2 by evaluating the objective function.
        INPUTS:
            U: (2M,) float, representing the coefficient vector.
            ipDT: (M^2-1,M^2-1) float, matrix where entry (m,n) is 
                    <\grad^\theta phi_m, \grad^\theta phi_n>_{\compMes}.
        OUTPUTS:
            Jval: float, the value of the objective function. By definition, this is real, and equal to lbda.
        '''
        
        _, ipDT = TLambdaStores(self.M, self.theta)
        return np.real( np.sum( np.outer(U, np.conjugate(U)) * ipDT ) )
        
    def val(self, x, y):
        '''
        Evaluates the Poly2D at the point(s) (x,y)
        INPUTS:
            x,y: (l,) floats, (x,y) coordinate pairs as 1D arrays.
        OUTPUTS:
            polyVals: (l,) complex, values of the Poly2D at the input co-ordinates.
        '''
        
        polyVals = np.zeros_like(x, dtype=complex)
        for m in range(self.M**2):
            jm = m % self.M
            im = m // self.M
            polyVals += self.uCoeffs[m] * (x**im) * (y**jm)
        return polyVals

    def plot(self, N=250, levels=15):
        '''
        Creates a heatmap of the function's real and imaginary parts, returning the figure handles for each.
        INPUTS:
            N: int, number of meshpoints to use for domain
            levels: int, number of contour levels to use
        OUTPUTS:
            rF, aF: figure handles, handles for heatmaps of the function over the region Omega.
        '''
        
        rF, rAx = plt.subplots()
        iF, iAx = plt.subplots()
        for a in [rAx, iAx]:
            a.set_aspect('equal')
            a.set_xlabel(r'$x_1$')
            a.set_ylabel(r'$x_2$')
        rAx.set_title(r'$\Re(\varphi)$, $\lambda=%.3f $' % (self.lbda))
        iAx.set_title(r'$\Im(\varphi)$, $\lambda=%.3f $' % (self.lbda))

        x = y = np.linspace(0,1,num=N)
        X, Y = np.meshgrid(x,y)
        U = self.val(X,Y)
        rCon = rAx.contourf(X, Y, np.real(U), levels=levels)
        iCon = iAx.contourf(X, Y, np.imag(U), levels=levels)
        # make colourbars
        rF.colorbar(rCon)
        iF.colorbar(iCon)
        return [rF, rAx], [iF, iAx]

### Saving the output of the Run

We will use the function `SaveRun` to save our results to `.npz` files which we can later read in.
The only important things to store are the values of the quasimomentum, and the array of coefficients - everything else, provided one has access to the `Poly2D` class, can be reconstructed.
The number of eigenfunctions can of course be computed from the dimensions of `uRealStore` of course (it's the number of rows that this array possesses).

In [14]:
def SaveRun(theta, uRealStore, fname='', fileDump='./Poly2D_Results/'):
    '''
    Saves the output of the minimisation proceedure (the coefficients) to a .npz file, 
    which can then be read in by another script or notebook.
    INPUTS:
        theta: (2,) float, value of the quasi-momentum.
        uRealStore: (n,2M^2) float, the coefficients of the Poly2D functions (in real format) found by the optimisation
        fname: str, the name of the file to which values will be saved.
        fileDump: str, path to the directory in which to save the .npz file.
    OUTPUTS:
        outFile: str, name of file to which values were saved
    '''
    
    M = np.sqrt( np.shape(uRealStore)[1] // 2 )
    N = np.shape(uRealStore)[0]
    
    if not fname:
        #automatically create file name
        timeStamp = datetime.now()
        outFile = fileDump + timeStamp.strftime("%Y-%m-%d_%H-%M_PolyCoeffsM-") + str(int(M)) + '.npz'
    else:
        outFile = fileDump + fname
    
    np.savez(outFile, theta=theta, uRealStore=uRealStore)
    return outFile

### Solution of the Discretised Problem

Our minimisation problem can now be solved through use of `scipy.optimize.minimize`, using the functions above to build the various constraints and inner-product information.

In [15]:
## Output files
fn = ''
fd = './Poly2D_Results/'

## Run parameters

# M-1 is highest order of the 2D polynomial approximation
M = 15
# We want to find the first N eigenvalues
N = 3
# Quasimomentum value
theta = np.zeros((2,))
theta[0] = 0.
theta[1] = np.pi/2

## Solver behaviour
maxIter = 2500
display = True

## Initialise storage for the solutions
# (l,2m) + (l,2m+1) is the coefficient u^l_m
uRealStore = np.zeros((N, 2*M*M), dtype=float)
omegaSqStore = np.zeros((N,), dtype=float)

# Compute inner products
ip, ipDT = TLambdaStores(M, theta)
# Set objective function
JandJacOpt = lambda UU: JandJac(UU, ipDT)
# Set initial guess to be the constant function
U0 = np.zeros((M*M,), dtype=complex)
U0[0] += 1./np.sqrt(3.)
UU0 = Comp2Real(U0)

## Solving begins

# solve each minimisation problem in sequence,
# starting with finding u^1, which adheres to no orthogonality conditions,
# up to finding u^n, which adheres to N-1 orthogonality conditions

# We can setup the norm constraint independently though
nonLinCon, _ = NormConstraint(M, ip)

print('Beginning solve [n]...')

# for ease, we find u^1 outside the loop
print(' ----- \n [0] ')
linCon, _ = BuildLinearConstraint(M,ip)
t0 = time.time()
resultJ = minimize(JandJacOpt, UU0, constraints=[linCon, nonLinCon], jac=True, options={'maxiter' : maxIter, 'disp' : display}, method='SLSQP')
t1 = time.time()
# save the coefficients for use in the next solve
uRealStore[0,:] = resultJ.x
# save the eigenvalue
omegaSqStore[0] = resultJ.fun
print('Runtime: approx %d mins (%s seconds) \n -----' % (np.round((t1-t0)//60), t1-t0))

# for every eigenfunction and value that we want above the first
for n in range(1,N):
    print(' ----- \n [%d]' % n)
    # setup and solve minimisation problem with n constraints
    linCon, _ = BuildLinearConstraint(M, ip, prevUs=uRealStore[:n,:])
    t0 = time.time()
    resultJ = minimize(JandJacOpt, UU0, constraints=[linCon, nonLinCon], jac=True, options={'maxiter' : maxIter, 'disp' : display}, method='SLSQP')
    t1 = time.time()
    # save coefficients
    uRealStore[n,:] = resultJ.x
    # save eigenvalue
    omegaSqStore[n] = resultJ.fun
    print('Runtime: approx %d mins (%s seconds) \n -----' % (np.round((t1-t0)//60), t1-t0))
    
SaveRun(theta, uRealStore, fname=fn, fileDump=fd)

Beginning solve [n]...
 ----- 
 [0] 
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5854496315477167
            Iterations: 38
            Function evaluations: 53
            Gradient evaluations: 38
Runtime: approx 0 mins (5.019851922988892 seconds) 
 -----
 ----- 
 [1]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 14.234086507624966
            Iterations: 76
            Function evaluations: 124
            Gradient evaluations: 76
Runtime: approx 0 mins (10.008551120758057 seconds) 
 -----
 ----- 
 [2]
Optimization terminated successfully    (Exit mode 0)
            Current function value: 23.010034504535845
            Iterations: 118
            Function evaluations: 560
            Gradient evaluations: 118
Runtime: approx 0 mins (16.935363292694092 seconds) 
 -----


'./Poly2D_Results/2021-11-29_12-02_PolyCoeffsM-15.npz'

In [16]:
plot=False

if plot:
    for n in range(N):
        P = Poly2D(theta, Real2Comp(uRealStore[n,:]))
        print(P)

In [17]:
test=False

if test:
    ### TESTING GROUND, I HAVE FAITH IN THE MACHINE WHEN THETA=0

    U = np.zeros((M*M,), dtype=complex)
    U[0] += 1.
    print(U)

    print('Think norm^2 is:', f(Comp2Real(U)))
    print('Think norm^2 of tgrad is:', Jopt(Comp2Real(U)))

    # bulk terms
    pmpn, l2mn, _, _, l2dtmdtn = Lambda2Stores(M, theta)
    # horz edge terms
    lhmn, _, _, lhdtmdtn = LambdaHStores(M, theta)
    # vert edge terms
    lvmn, _, _, lvdtmdtn = LambdaVStores(M, theta)

    # now, reapply pmpn to each term and construct the norms in the whole space
    # remember, we need to replace pmpn in our terms now
    # NOTE: If we aren't treating p_m=1 for each pm, you'll need to uncomment the multiplication by pmpn
    ip = ( l2mn + lhmn + lvmn ) #* pmpn
    ipDT = ( l2dtmdtn + lhdtmdtn + lvdtmdtn ) #* pmpn